# Talking to CSV and Excel files with LangChain

In [ ]:
!pip -q install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.3/426.3 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 1.8 MB/s eta 0:00:00


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.123
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: aiohttp, dataclasses-json, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Dataset - Black Friday Sales


https://www.kaggle.com/datasets/sdolezel/black-friday

In [ ]:
!wget -q https://www.dropbox.com/s/8y6a1zloiscuo5d/black_friday_sales.zip
!unzip -q black_friday_sales.zip

In [ ]:
import pandas as pd

df = pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


## CSV Agent

NOTE: this agent calls the Pandas DataFrame agent under the hood, which in turn calls the Python agent, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.

In [ ]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

In [ ]:
agent = create_csv_agent(OpenAI(temperature=0), 
                         '/content/train.csv', 
                         verbose=True)

In [ ]:
agent

AgentExecutor(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f68343c37c0>, verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f68343c37c0>, verbose=False, prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={'df': '   User_ID Product_ID Gender   Age  Occupation City_Category  \\\n0  1000001  P00069042      F  0-17          10             A   \n1  1000001  P00248942      F  0-17          10             A   \n2  1000001  P00087842      F  0-17          10             A   \n3  1000001  P00085442      F  0-17          10             A   \n4  1000002  P00285442      M   55+          16             C   \n\n  Stay_In_Current_City_Years  Marital_Status  Product_Category_1  \\\n0                          2               0                   3   \n1                          2               0             

In [ ]:
agent.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'


You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`:
{df}

Begin!
Question: {input}
{agent_scratchpad}

In [ ]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows
Action: python_repl_ast
Action Input: len(df)
Observation: 550068
Thought: I now know the final answer
Final Answer: 550068

> Finished chain.


'550068'

In [ ]:
agent.run("how many people are female?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of people who are female
Action: python_repl_ast
Action Input: df[df['Gender'] == 'F'].count()
Observation: User_ID                       135809
Product_ID                    135809
Gender                        135809
Age                           135809
Occupation                    135809
City_Category                 135809
Stay_In_Current_City_Years    135809
Marital_Status                135809
Product_Category_1            135809
Product_Category_2             91530
Product_Category_3             37594
Purchase                      135809
dtype: int64
Thought: I now know the final answer
Final Answer: 135809

> Finished chain.


'135809'

In [ ]:
agent.run("how many people have stayed more than 3 years in the city?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to find the people who have stayed more than 3 years in the city
Action: python_repl_ast
Action Input: df[df['Stay_In_Current_City_Years'] > '3'].shape[0]
Observation: 84726
Thought: I now know the final answer
Final Answer: 84726 people have stayed more than 3 years in the city.

> Finished chain.


'84726 people have stayed more than 3 years in the city.'

In [ ]:
agent.run("how many people have stayed more than 3 years in the city and are female?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to find the people who meet the criteria
Action: python_repl_ast
Action Input: df[(df['Stay_In_Current_City_Years'] > '3') & (df['Gender'] == 'F')]
Observation:         User_ID Product_ID Gender    Age  Occupation City_Category  \
29      1000010  P00085942      F  36-45           1             B   
30      1000010  P00118742      F  36-45           1             B   
31      1000010  P00297942      F  36-45           1             B   
32      1000010  P00266842      F  36-45           1             B   
33      1000010  P00058342      F  36-45           1             B   
...         ...        ...    ...    ...         ...           ...   
549972  1005901  P00372445      F  36-45           7             C   
549981  1005911  P00372445      F  26-35          17             C   
550011  1005952  P00371644      F  46-50           1             C   
550065  1006036  P00375436      F  26-35          15       

'18596 people have stayed more than 3 years in the city and are female.'

In [ ]:
agent.run("Are there more males or females?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of males and females
Action: python_repl_ast
Action Input: df['Gender'].value_counts()
Observation: M    414259
F    135809
Name: Gender, dtype: int64
Thought: I now know the final answer
Final Answer: There are more males than females.

> Finished chain.


'There are more males than females.'

## LangChain CSV Loader

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/content/train.csv')
data = loader.load()

In [ ]:
#custom chain etc

## Convert XLS file to csv

In [ ]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget -q https://www.dropbox.com/s/co5kwipba9hpumt/excel_files.zip
!unzip -q excel_files.zip

In [ ]:
xls_file = r'/content/excel_file_example.xlsx'
output_csv = r'/content/excel_file_example.csv'

# Read the XLS file using pandas and openpyxl as the engine
data = pd.read_excel(xls_file, engine='openpyxl')

# Save the data as a CSV file
data.to_csv(output_csv, index=False)

In [ ]:
agent = create_csv_agent(OpenAI(temperature=0), 
                         '/content/excel_file_example.csv', 
                         verbose=True)

In [ ]:
agent.run("What are the column names?")



> Entering new AgentExecutor chain...
Thought: I need to find out what the column names are
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['0', 'First Name', 'Last Name', 'Gender', 'Country', 'Age', 'Date',
       'Id'],
      dtype='object')
Thought: I now know the column names
Final Answer: 0, First Name, Last Name, Gender, Country, Age, Date, Id

> Finished chain.


'0, First Name, Last Name, Gender, Country, Age, Date, Id'

In [ ]:
agent.run("What is the average age?")



> Entering new AgentExecutor chain...
Thought: I need to calculate the average age of the people in the dataframe
Action: python_repl_ast
Action Input: df['Age'].mean()
Observation: 33.26
Thought: I now know the final answer
Final Answer: The average age is 33.26.

> Finished chain.


'The average age is 33.26.'

In [ ]:
agent.run("Which country appears the most and how many times does it appear?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of times each country appears in the dataframe.
Action: python_repl_ast
Action Input: df['Country'].value_counts()
Observation: United States    48
Great Britain    28
France           24
Name: Country, dtype: int64
Thought: I now know the final answer
Final Answer: The country that appears the most is United States, appearing 48 times.

> Finished chain.


'The country that appears the most is United States, appearing 48 times.'

In [ ]:
agent.run("What is the ratio of males to females?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of males and females
Action: python_repl_ast
Action Input: df['Gender'].value_counts()
Observation: Female    76
Male      24
Name: Gender, dtype: int64
Thought: I now know the final answer
Final Answer: 24:76

> Finished chain.


'24:76'